Plotting some figures for John's Thesis Writing class

In [1]:
import sys

import numpy as np
import netCDF4 as nc
import numpy.matlib
import datetime
import xarray as xr
from scipy import interpolate
from numpy import ma
from scipy import stats
import scipy.io as sio
import pickle as pickle
from sklearn import linear_model
import numpy.ma as ma
import matplotlib.patches as mpatches
from shapely.geometry.polygon import LinearRing

import scipy as sp
import pandas as pd

import time

from copy import copy 

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import ticker

from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1.axes_divider import HBoxDivider
import mpl_toolkits.axes_grid1.axes_size as Size
from mpl_toolkits.axes_grid1 import make_axes_locatable

# OS interaction
import os
import sys
import cftime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point

from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

import matplotlib.colors as mcolors

import glob
import dask
import dask.bag as db

from scipy import interpolate

import statsmodels.stats.multitest as multitest

from matplotlib.gridspec import GridSpec
from cartopy.crs import EqualEarth, PlateCarree

import calendar

In [2]:
data_in_path = '/glade/u/home/zcleveland/scratch/ERA5/dsw/' # path to subsetted data
fig_out_path = '/glade/u/home/zcleveland/NAM_soil-moisture/ERA5_analysis/figures/ATMOS_6600/' # path to generated figures
temp_scratch_path = '/glade/u/home/zcleveland/NAM_soil-moisture/ERA5_analysis/temp/' # path to temp directory in scratch

Plot the average onset day of the NAM

In [ ]:
# open dataset
ds = xr.open_dataset(os.path.join(data_in_path, 'NAM_onset_mean.nc'))
da = ds['dayofyear']

# plot
# set projection and subplot details
projection = ccrs.PlateCarree()    
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': projection})

v_min = 150
v_max = 220

# set levels and ticks for plotting
levels = np.arange(v_min, v_max+1, 1)

# plot the data using contourf
cf = ax.contourf(da.longitude, da.latitude,
                  da, cmap='RdBu', levels=levels, vmin=v_min, vmax=v_max)    

# plot contour lines
contour_levels = np.arange(v_min, v_max+1, 10)
cs = ax.contour(da.longitude, da.latitude, da,
                levels=contour_levels, colors='black', linestyles='dashed')
# add labels for contour lines
plt.clabel(cs, inline=True, fmt='%d', fontsize=8)

# add coastlines, state borders, and other features
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES)

# add colorbar and ticks
cbar = plt.colorbar(cf, ax=ax, ticks=np.arange(v_min, v_max+1, 10))
cbar.set_label('Ordinal Day')

# add title and others
plt.title(f'Average Onset Day of the NAM')
plt.tight_layout()
# plt.savefig(f'{fig_out_path}onset_day_mean.png', dpi=1000)
plt.show()
plt.close()

Plot the correlation between springtime snow depth and onset

In [ ]:
# set variable
var = 'sd'

# create list of months over which to average
var_mons = [3,4,5]

# open onset dataset
onset_ds = xr.open_dataset(f'{data_in_path}NAM_onset.nc')
onset_ds['year'] = onset_ds['year'].dt.year # convert to only year.  e.g. 2012-01-01 -> 2012
onset_da = onset_ds['date']
onset_data = onset_da.dt.dayofyear.astype('float32')

# open snow depth dataset
var_files = glob.glob(f'{data_in_path}*/{var}*.nc')
var_ds = xr.open_mfdataset(var_files)

# pull out actual variable name in the dataset since they can be different names/capitalized
var_name = [v for v in var_ds.data_vars.keys() if f'{var[0].upper()}' in v][0]
var_da = var_ds[var_name]

# get data from var
if 'AVG' in var_name:
    mon_mean = var_da.resample(time='1M').mean()
    var_mon_mean = mon_mean.sel(time=mon_mean['time.month'].isin(var_mons))
    var_data = var_mon_mean.groupby('time.year').mean(dim='time')
else:
    mon_sum = var_da.resample(time='1M').sum()
    var_mon_sum = mon_sum.sel(time=mon_sum['time.month'].isin(var_mons))
    var_data = var_mon_sum.groupby('time.year').sum(dim='time')

# calculate correlation
var_corr = xr.corr(onset_data, var_data, dim='year')

# plot the results
projection = ccrs.PlateCarree()    
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw=dict(projection=projection))

# plot the data using contourf
cf = plt.contourf(var_corr.longitude, var_corr.latitude,
                  var_corr, levels=np.arange(-1, 1.1, .1),
                  cmap='RdBu_r', vmin=-1, vmax=1)    

# add coastlines, state borders, and other features
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES)

# make string for month letters from var_range (e.g. [6,7,8] -> 'JJA')
var_months = ''.join([calendar.month_name[m][0] for m in var_mons])

plt.title(f'Correlation Between Start Date of the Monsoon \nand Spring Snow Depth ({var_months})')    
plt.colorbar(cf, ax=ax, label='Pearson Correlation Coefficient')
plt.savefig(f'{fig_out_path}corr_onset_{var}-{var_months}.png', dpi=1000)
plt.show()
plt.close()
